<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.975 · Deep Learning · PAC1</p>
<p style="margin: 0; text-align:right;">2024-2 · Màster universitari en Ciència de dades (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudis de Informàtica, Multimèdia y Telecomunicació</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>

# PEC 1: Xarxes neuronals artificials i convolucionals amb Keras - Classificació d’imatges i superresolució

Al llarg d'aquesta pràctica, implementarem diversos models de xarxes neuronals, utilitzant Keras i la base de dades Imagenette (versió 320px). Concretament, abordarem les tasques següents:
1. Descàrrega, anàlisi i pre-processament de les dades (1,5 pts)
2. Xarxa neuronal artificial (ANN) completament connectada (1,5 pts)
3. Petita xarxa neuronal convolucional (CNN) (2 pts)
4. Augmentació de dades (1 pt)
5. Superresolució d’imatges amb CNNs (1,5 pts)
6. Inferència, mètriques i visualització dels resultats (1 pt)
7. Model Pre-entrenat en Superresolució (1 pt)
8. Comparació i conclusions (0,5 pts)

<u> Consideracions generals </u>:

- La solució proposada no pot utilitzar mètodes, funcions o paràmetres declarats **_deprecated_** en versions futures.
- Aquest PAC s’ha de fer d’una manera **estrictament individual**. Qualsevol indici de còpia serà penalitzat amb un suspens (D) per a totes les parts implicades i la possible avaluació negativa de la totalitat de l'assignatura.
- Cal que l'estudiant indiqui **totes les fonts** que ha utilitzat per a la realització del PEC. En cas contrari, es considerarà que l’alumne ha comès el plagi, sent penalitzat amb un suspens (D) i la possible avaluació negativa de la totalitat de l'assignatura.
- Si s’utilitza qualsevol **IA generativa** en la resolució de la PAC **s’ha de referenciar** en aquelles seccions on s’ha utilitzat, com qualsevol altra font.

<u>Format del lliurament </u>:

- Alguns exercicis poden suposar diversos minuts d’execució, de manera que el lliurament s’ha de fer en format **Notebook** i en el format **html**, on es vegi el codi, els resultats i els comentaris de cada exercici. Podeu exportar el quadern a HTML a Jupyter Notebook des del menú File $\to$ Download as $\to$ HTML.
- Hi ha un tipus especial de cel·la per allotjar el text. Aquest tipus de cel·les serà molt útil per respondre a les diferents preguntes teòriques plantejades al llarg de l’activitat. Per canviar el tipus de cel·la a aquest tipus, al menú: Cell $\to$ Cell Type $\to$ Markdown.

## 0. Context i càrrega de llibreries
Al llarg d’aquesta pràctica, implementarem diversos models de xarxes neuronals per classificar les imatges de la base de dades [Imagenette](https://github.com/fastai/imagenette).

La base de dades Imagenette és un subconjunt de 10 classes fàcilment classificables de [Imagenet](https://www.image-net.org/), un projecte fonamental per avançar en la investigació sobre visió artificial i aprenentatge profund. Imagenette conté unes 13.000 imatges de diferents mides pertanyents a 10 categories (tench, English springer, cassette player, chain saw, church, French horn, garbage truck, gas pump, golf ball, parachute), cadascuna en una carpeta diferent.

Concretament en aquesta PAC, utilitzarem una versió (Imagenette2-320) que ha estat re-escalada, però mantenint la relació d’aspecte de cada imatge (s’han ajustat de manera que la dimensió menor de cada imatge sigui de 320 píxels). Això atenuarà la càrrega computacional dels algoritmes quan s’utilitzen bases de dades d’imatges, però mantenint una qualitat suficient necessària per als nostres experiments. Les dades venen separades en 2 conjunts, entrenament i validació.

**Nota: A causa de l’ús d’imatges com dades d’aquesta pràctica, l’entrenament de cada exercici es pot retardar entre uns minuts i més de mitja hora mitjançant GPU (els temps utilitzant CPU són significativament més llargs). Es recomana realitzar la pràctica en l’entorn que ofereix la plataforma Kaggle, ja que ofereix un entorn gratuït amb 30 hores setmanals per a l’ús de la GPU.**

Al llarg de tota la pràctica, per a la creació de les diferents xarxes, alternem l’ús del model [Sequential](https://keras.io/guides/sequential_model/) i el model [Functional](https://keras.io/guides/functional_api/) de Keras a través de les seves classes [Sequential](https://keras.io/api/models/sequential/) i [Model](https://keras.io/api/models/model/) respectivament.

Es recomana la lectura detallada de la documentació dels dos models per dur a terme la realització de la pràctica.

Comencem per instal·lar i carregar les llibreries més rellevants:

In [1]:
# Instal·lem la darrera versió de Tensorflow (amb CUDA)
%pip install tensorflow[and-cuda]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.6/412.6 MB 4.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 105.4 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.0/22.0 MB 84.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 65.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.7/867.7 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 704.7/704.7 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.8/98.8 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 MB 5.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.5/197.5 MB 8.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 10.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━

In [ ]:
# Importem Tensorflow
import tensorflow as tf
print("TensorFlow version   : ", tf.__version__)

# Necessitarem GPU
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

# Keras versió is 3.5.0
from tensorflow import keras
print("Keras version   : ", keras.__version__)

In [ ]:
# Importem els elements de Keras que utilitzarem amb més freqüència
from keras.utils import image_dataset_from_directory
from keras.layers import (Input, Dense, Dropout, Flatten, Conv2D, Conv2DTranspose,
                          MaxPooling2D, UpSampling2D, Rescaling, Resizing,
                          RandomFlip, RandomRotation)
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam

In [ ]:
# Importem algunes llibreries que necessitarem per a la PAC
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from pathlib import Path

## 1. Descàrrega, anàlisi i dades pre-processades (1,5 punts)

En aquesta secció explorarem la base de dades i prepararem la càrrega de les imatges per als models de les seccions següents.

Per crear la nostra base de dades, hem de descarregar el fitxer d’imatges del següent [enllaç](https://drive.google.com/file/d/16kqYxjXK0xDDYQ4xF3PBgsa0NG3gwT4L/view?usp=sharing) (és un fitxer .zip que ocupa aproximadament 340 Mb).

**Nota: per descarregar el fitxer d'imatges, heu de iniciar la sessió amb l'usuari i la contrasenya de la UOC.**

A partir d’aquí:

- Si treballem en local, simplement hem de descomprimir el fitxer descarregat.

- Si treballem des de Kaggle hem de pujar el notebook de l'enunciat a la plataforma (per això podeu seguir els 6 primers passos del següent [article](https://rajputankit22.medium.com/how-to-upload-my-own-notebook-to-kaggle-2b0dedbb5a6b)) i després, un cop pujat el notebook, expandir la barra lateral desplegable de la dreta i al menú 'Input' clickar el botó 'Upload' i pujar el fitxer descarregat prèviament. Després cal donar un nom a la base de dades i quan es carregui el fitxer ja tindreu accessible la base de dades a la ruta <code> ../ input/</code>.

Un cop tinguem la base de dades accessible, la inspeccionarem.

A la carpeta <code>/images</code> (si treballem a casa) o <code>/kaggle/input/nom-base-de-dades/images</code> (si treballem des de Kaggle) trobem 2 carpetes:
- A la carpeta <code>/train</code> es troba el total de les imatges d'entrenament separades per classes (cada classe en una carpeta diferent).
- A la carpeta <code>/val</code>, es troba el total de les imatges de validació separades per classes (cada classe en una carpeta diferent).

Com podem veure, tenim imatges per dur a terme l'entrenament i la validació dels models, però no tenim un conjunt de prova, el crearem durant aquesta primera secció.

Comencem per obtenir les dades i analitzar la seva estructura i característiques.
### 1.1. Anàlisi dels fitxers d'imatges

Primer, inspeccionarem l’organització de les dades.

<div style = "background-color: #edf7ff; border-color: #7c9dbf; frontera-esquerra: 5px sòlid #7c9dbf; encoixinat: 0,5EM;"> <strong> Exercici 1.1 [0,5 pts]: </strong> A partir de l'estructura de carpetes indicada i per a cada conjunt de dades (<code>/train</code> i <code>/val</code>):
    <ul>
        <li> Extreu els noms de les 10 classes de cada conjunt i comproveu que siguin iguals.</li>
        <li> Obteniu quantes instàncies hi ha en total per a cada classe i representa la distribució en un diagrama de barres.</li>
        </ul>
Respon també les preguntes següents:
    <ul>
        <li> Quin percentatge de les imatges totals corresponen a cada conjunt (train/val)?</li>
        <li> Segons els diagrames de barres, podem dir que la distribució per classes és similar en els dos conjunts?</li>
        <li> Pel que fa al nombre d’imatges per classe, les classes estan equilibrades o hi ha desequilibris notables?</li>
    </ul>
<strong> Nota: es valorarà la concisió en la resposta a les preguntes (una o dues frases són suficients per a cadascuna de les respostes).</strong>
</div>

In [ ]:
import os
from pathlib import Path

# Directoris (ajustar la ruta segons ubicació de les dades)
dataset_path = '/kaggle/input/m2-975-pac1-images/images'
# List directories
for folder in sorted(os.listdir(dataset_path)):
    path = os.path.join(dataset_path, folder)
    if os.path.isdir(path):
        print(f"📂 {folder}")
        for subfolder in sorted (os.listdir(path)):
            print("  " + f"📂 {subfolder}")

In [ ]:
# Llistar classes a cada conjunt
val_classes = os.listdir('/kaggle/input/m2-975-pac1-images/images/val')
print(val_classes)
train_classes = os.listdir('/kaggle/input/m2-975-pac1-images/images/train')
print(train_classes)

In [ ]:
# Verificar que les dues llistes de classes són iguals
[os.listdir('/kaggle/input/m2-975-pac1-images/images/val')]==[os.listdir('/kaggle/input/m2-975-pac1-images/images/train')]

In [ ]:
val_path = Path('/kaggle/input/m2-975-pac1-images/images/val')
train_path = Path('/kaggle/input/m2-975-pac1-images/images/train')

# Imprimeix nombre d'imatges per classe a train
count_train = {}
for f in sorted (os.listdir(train_path)):
    count_train[f]=len(os.listdir(os.path.join(train_path, f)))
print(count_train)

# Imprimeix nombre d'imatges per classe a val
count_val = {}
for f in sorted (os.listdir(val_path)):
    count_val[f]=len(os.listdir(os.path.join(val_path, f)))
print(count_val)

# Comptar total d'imatges per classe
count_total = count_train.copy()
for k, v in count_val.items():
        count_total[k]+=v
print(count_total)

# Comptar total d'imatges per conjunt
count_train_total = sum(count_train.values())
count_val_total = sum(count_val.values())
count_total_values = sum(count_total.values())
print(f" El nombre total d'imatges per a l'entrenament és de: {count_train_total}")
print(f" El nombre total d'imatges per al test és de: {count_val_total}")
print(f" El nombre total d'imatges és de: {count_total_values}")


In [ ]:
# Representació gràfica de la distribució de classes
val=plt.bar(count_val.keys(), count_val.values(), label='Test', color='skyblue')
train=plt.bar(count_train.keys(), count_train.values(), bottom=list(count_val.values()), label = 'Train', color='lightcoral')
plt.xlabel('Class')
plt.ylabel('Number of images')
plt.title('Total images per class')
plt.legend()
plt.bar_label(val,label_type='center')
plt.bar_label(train,label_type='center')
plt.tick_params(axis='x', labelrotation=90)
plt.show()

# Referències
# https://www.datacamp.com/es/tutorial/python-bar-plot
# https://stackoverflow.com/questions/10998621/rotate-axis-tick-labels

<div style="background-color: #fcf2f2; border-color: #dfb5b4; border-left: 5px solid #dfb5b4; padding: 0.5em;">
<strong> Comentaris: </strong>
<br> <br>
        <ul>
        <li> <strong> Quin percentatge de les imatges totals correspon a cada conjunt (train/val)?</strong> 29,3% val i 70,7% train</li>
        <li> <strong> Segons els diagrames de barres, podem afirmar que la distribució per classes és similar en els dos conjunts?</strong> És força similar en totes les classes.</li>
        <li> <strong> Pel que fa al nombre d’imatges per classe, les classes estan equilibrades o hi ha desequilibris notables?</strong> No hi ha desequilibris notables</li>
    </ul>     
<br> <br>
</div>

### 1.2. Visualització d’imatges.

Ara examinarem el format de les imatges per entendre la seva mida i el seu rang de valors. Visualitzarem algunes imatges d’exemple de cada classe.

<div style = "background-color: #edf7ff; border-color: #7c9dbf; frontera-esquerra: 5px sòlid #7c9dbf; encoixinat: 0,5EM;"> <strong> Exercici 1.2 [0,5 pts]: </strong> Mostra com a exemple una imatge per a cada categoria del conjunt d'entrenament. A continuació, respon:
    <ul>
        <li> Tenen totes les imatges les mateixes dimensions (alçada × amplada)?
        <li> Quin és el rang dinàmic dels píxels (valors mínim i màxim) a les imatges?
        <li> Aquests valors són adequats per entrenar una xarxa neuronal?
        <li> Quines accions de pre-processament serien necessàries en relació a la pregunta anterior?
    </ul>
<strong> Nota: es valorarà la concisió en la resposta a les preguntes (una o dues frases són suficients per a cadascuna de les respostes).</strong>
</div>

In [ ]:
import random
from PIL import Image

train_path = '/kaggle/input/m2-975-pac1-images/images/train'
collage = Image.new("RGBA", (1000,4000))
img_h = 0
for f in sorted (os.listdir(train_path)):
    img = Image.open(os.path.join(train_path, f,random.choice(os.listdir(os.path.join(train_path, f)))))
    collage.paste(img, (0,img_h))
    img_h+=img.size[1]
collage

# Referències
# https://docs.python.org/3/library/random.html
# https://pillow.readthedocs.io/en/stable/reference/Image.html

In [ ]:
from itertools import islice
min_width = 1000
min_height = 1000
min_rgb = [0,0,0]
max_rgb = [0,0,0]
for folder in islice(sorted (os.listdir(train_path)),0,3):
    for file in islice(sorted (os.listdir(os.path.join(train_path, folder))),0,3):
        img = Image.open(os.path.join(train_path, folder,file))
        width, height = img.size
        if ((width != min_width) or (height != min_height)):
            min_width = min(min_width, width)
            min_height = min(min_height, height)
        pixels = list(img.getdata())
        for pixel in pixels:
                min_rgb = (min(min_rgb[0], pixel[0]), min(min_rgb[1], pixel[1]), min(min_rgb[2], pixel[2]))
                max_rgb = (max(max_rgb[0], pixel[0]), max(max_rgb[1], pixel[1]), max(max_rgb[2], pixel[2]))
print(min_width, min_height, min_rgb, max_rgb)

<div style="background-color: #fcf2f2; border-color: #dfb5b4; border-left: 5px solid #dfb5b4; padding: 0.5em;">
<strong> Comentaris: </strong>
<br> <br>
        <ul>
        <li> <strong> Mida </strong>: Les imatges tenen una mida diferent. </li>
        <li> <strong> Rang dinàmic </strong>: Per a cada canal els valors mínim i màxim són 0 i 255 respectivament. </li>
        <li> <strong> Adequació per a la xarxa </strong>: No són adequades. </li>
        <li> <strong> Accions necessàries </strong>: Per tal d'adequar les dades als requeriments de la xarxa convé uniformitzar la mida de les imatges i normalitzar els valors dels píxels. </li>
    </ul>
</div>

### 1.3. Creació dels conjunts de dades en format Keras/TensorFlow

A continuació, prepararem les dades per a l'entrenament amb Keras. Utilitzarem la funció <code>**tf.keras.utils.image_dataset_from_directory()**</code> de TensorFlow/Keras, que permet crear lots de dades etiquetats en funció dels directoris d'imatges organitzats per classe.

La documentació d'aquesta funció es troba tant al lloc web de [Keras](https://keras.io/api/data_loading/image/) com al de [TensorFlow](https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory).

Aquesta funció ens facilitarà generar conjunts d'entrenament, validació i prova a partir de les carpetes analitzades. Les imatges es re-dimensionaran a una mida fixa i s’organitzaran en lots (batch).

**Especificacions**: convertirem les imatges a la mida de 160 × 160 píxels i les agruparem en lots de 64 imatges. També separarem part de les dades de prova a partir de les de validació.


<div style = "background-color: #edf7ff; border-color: #7c9dbf; fronter-esquerra: 5px sòlid #7c9dbf; encoixinat: 0,5EM;"> <strong> Exercici 1.3 [0,5 pts]: </strong> Utilitzeu la funció <code>tf.keras.utils.image_dataset_from_directory () </code> per generar 3 conjunts de dades a partir de les carpetes <code>/train</code> i <code>/val</code>:
<ul>
  <li><strong>train_ds</strong>: Conjunt d'entrenament a partir de <code>/train/</code>. Redimensiona les imatges a 160×160, amb <code>batch_size=64</code> i <code>label_mode="categorical"</code> (10 categories).</li>
  <li><strong>val_ds</strong>: Conjunt de validació a partir del directori <code>/val/</code>, amb un <code>validation_split</code> de 0.5 usant <code>subset="validation"</code> a la funció i fixa un <code>seed</code> per a reproducibilitat. De nou redimensiona les imatges a 160x160 amb <code>batch_size=64</code> i <code>label_mode="categorical"</code>.</li>
  <li><strong>test_ds</strong>: Conjunt de test a partir del mateix directori <code>/val/</code> usant <code>subset="training"</code> a la funció (amb el mateix <code>seed</code> per obtenir la partició complementària). Redimensiona també a 160×160 amb <code>batch_size=64</code> i fes servir <code>label_mode="categorical"</code>.</li>
 </ul>
<strong>NOTA</strong>: l'assignació de <code>subset="validation"</code> i <code>subset="training"</code> és completament arbitrària i podria haver-se fet al revés. L'important és dividir les dades que es troben a la carpeta <code>/val</code> al 50% entre validació i test.
 <br><br>
Comproveu que la base de dades s'ha creat correctament imprimint per pantalla els noms de les classes i les dimensions de cada conjunt de dades.

Respon les preguntes següents:
 <ul>
  <li>Quin percentatge aproximat de les imatges s'utilitza a cadascun dels 3 conjunts?</li>
  <li>Quins altres percentatges se solen utilitzar?</li>
 </ul>
<strong>NOTA: Es valorarà la concisió en la resposta a les preguntes (una o dues frases són suficients per cadascuna de les respostes).</strong>
</div>

In [ ]:
img_height, img_width = 160, 160
batch_size = 64

# Conjunt d'entrenament
train_ds = tf.keras.utils.image_dataset_from_directory('/kaggle/input/m2-975-pac1-images/images/train',image_size = (img_height, img_width),
batch_size = batch_size, label_mode='categorical')

In [ ]:
# Conjunt de validació i test (usem el directori de validació d'Imagenette també per a test)
val_ds, test_ds = tf.keras.utils.image_dataset_from_directory('/kaggle/input/m2-975-pac1-images/images/val',image_size = (img_height, img_width),
batch_size = batch_size, validation_split = 0.5, label_mode='categorical', subset = 'both', seed = 1)

In [ ]:
# Comprovació dels resultats
print(train_ds.class_names)
print(len(train_ds))
print(test_ds.class_names)
print(tf.data.experimental.cardinality(test_ds).numpy())
print(val_ds.class_names)
print(val_ds.cardinality().numpy())

<div style="background-color: #fcf2f2; border-color: #dfb5b4; border-left: 5px solid #dfb5b4; padding: 0.5em;">
<strong>Comentaris:</strong>
<br> <br>
        <ul>
        <li> <strong>Percentatge d'imatges per conjunt</strong>: train_ds (~70%), test_ds (~15%), val_ds (~15%)</li>
        <li> <strong>Altres percentatges</strong>: Es solen utilitzar percentatges de ~60-80% per entrenament, ~10-20% per validació i ~10-20% per test.</li>
        </ul>
<br>
<strong>Referències:</strong>
<br> <br>
    <ul>
        <li>https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image_dataset_from_directory </li>
        <li>https://www.tensorflow.org/api_docs/python/tf/data/Dataset</li>
        <li>https://www.tensorflow.org/tutorials/load_data/images</li>
        <li>https://www.tensorflow.org/tutorials/keras/classification</li>
    </ul>
</div>

## 2. Xarxa neuronal artificial (ANN) completament connectada (1,5 pts)

Com a primer model, entrenarem una xarxa neuronal **completament connectada** (xarxa densa o <it>Multilayer Perceptron</it>) per **classificar les imatges**. Això servirà de model de referència (línia de base). Atès que la gestió d’imatges de 160×160 píxels directament amb capes denses donaria lloc a vectors d’entrada molt grans i, per tant, un gran nombre de paràmetres, primer aplicarem una reducció de la dimensionalitat de les imatges d’entrada. Concretament, reduirem cada imatge a 32×32 píxels (mitjançant una capa de redimensionat) abans d’aplanar-la, que simplificarà el model.

**Arquitectura proposada**: Utilitzarem l’API funcional de Keras (classe Model) per construir la xarxa. Utilitzarem les capes Resizing i Rescaling de Keras per preparar les imatges, seguides de Flatten per aplanar-les, i diverses capes Dense intercalades amb Dropout pel classificador.

En aquesta secció utilitzarem les capes [Resizing](https://keras.io/api/layers/preprocessing_layers/image_preprocessing/resizing/), [Rescaling](https://keras.io/api/layers/preprocessing_layers/image_preprocessing/rescaling/), [Flatten](https://keras.io/api/layers/reshaping_layers/flatten/), [Dense](https://keras.io/api/layers/core_layers/dense/) i [Dropout](https://keras.io/api/layers/regularization_layers/dropout/) de Keras.


<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;"> <strong>Exercici 2.1 [0,5 pts]:</strong> Implementa un model <strong>funcional</strong> de Keras amb les següents característiques:
<ul>
   <li>Una capa que redueixi les dimensions d'entrada de (160,160) a (32,32).</li>
   <li>Una capa de reescalat perquè els valors de píxel quedin entre 0 i 1.</li>
   <li>Una capa Flatten per convertir la imatge reduïda en un vector 1D.</li>
   <li>Una capa densa completament connectada que tingui un nombre de neurones equivalent a dos terços de la mida de la capa anterior, activació ReLU.</li>
   <li>Una capa Dropout amb probabilitat 0.5.</li>
   <li>Una altra capa densa de la meitat de la mida que la capa densa inmediatament anterior, activació ReLU.</li>
   <li>Una altra capa Dropout amb probabilitat 0.5.</li>
   <li>Una capa de sortida densa amb la mida i la funció d'activació adequades per al problema de classificació que es planteja.</li>
 </ul>
<strong>Nota: Visualitzar els models de tots els exercicis amb la funció <code>summary()</code></strong>
<br><br>
Contestar les preguntes següents:
 <ul>
     <li>Quina mida hauria de tenir la capa d'entrada si haguéssim treballat amb les imatges de la base de dades directament de 160x160 píxels?</li>
     <li>Quina reducció s'aconsegueix en reescalar les imatges a 32x32 píxels?</li>
      <li>Quants paràmetres té el model?</li>
 </ul>

<strong>NOTA: Es valorarà la concisió en la resposta a les preguntes (una o dues frases són suficients per a cadascuna de les respostes).</strong>
</div>

In [ ]:
# Construcció del model ANN completament connectat (model funcional)
def get_uncompiled_model():
    num_classes = 10
    model = keras.Sequential([
        keras.layers.InputLayer(shape=(img_height, img_width, 3)),
        keras.layers.Resizing(32, 32),
        keras.layers.Rescaling(1./255),
        keras.layers.Flatten(),
        # La mida de la Flatten és 32 x 32 x 3 (RGB)
        keras.layers.Dense(2048, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(1024, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(num_classes, activation='softmax')  # Sortida amb 10 classes
    ])
    return model
    
model = get_uncompiled_model()
model.summary()
keras.utils.plot_model(model, "model.png", show_shapes=True,dpi=64)

In [ ]:
# Càlcul del nombre de paràmetres
size_flatten_layer = 32*32*3+1
print(size_flatten_layer)
param_dense1_layer = size_flatten_layer*2048
print(param_dense1_layer)
param_dense2_layer = (2048+1)*1024
print(param_dense2_layer)
param_dense3_layer = (1024+1)*10
print(param_dense3_layer)
param_total = param_dense1_layer + param_dense2_layer + param_dense3_layer
print(param_total)


<div style="background-color: #fcf2f2; border-color: #dfb5b4; border-left: 5px solid #dfb5b4; padding: 0.5em;">
<strong>Comentaris:</strong>
<br> <br>
        <ul>
        <li><strong>Quina mida hauria de tenir la capa d'entrada si haguéssim treballat amb les imatges de la base de dades directament de 160x160 píxels?</strong> Hauria de tenir una mida de 160x160x3 = 76800</li>
     <li><strong>Quina reducció s'aconsegueix en reescalar les imatges a 32x32 píxels?</strong> S'aconsegueix una reducció del 96%</li>
      <li><strong>Quants paràmetres té el model?</strong> Té 8.4 mil·lions de paràmetres.</li>
        </ul>
<br>
    <strong>Referències:</strong>
<br>
    <li>https://www.tensorflow.org/tutorials/load_data/images</li>
    <li>https://keras.io/api/layers/preprocessing_layers/image_preprocessing/resizing/</li>
    <li>https://keras.io/api/layers/core_layers/dense/</li>
    <li>https://keras.io/api/layers/reshaping_layers/flatten/</li>
    <li>https://keras.io/api/layers/regularization_layers/dropout/</li>
</div>

Procedim a compilar i entrenar el model:
<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;"> <strong> Exercici 2.2 [1 PTS]: </strong> Quan entreneu qualsevol model, és essencial fer una bona elecció dels hiperparameters. En aquest exercici compilarem i entrenarem el model creat analitzant els efectes del <em>learning rate</em> en l'entrenament. Concretament, provarem els valors de 1e-3, 1e-4 i 1e-5.

Per a cada valor de <em>learning rate</em> es demana:
<ul>
   <li> Compileu el model amb l'optimitzador <strong>Adam</strong> i el corresponent <em>learning rate</em>.</li>
   <li>
        Entrena el model durant <strong>100 epochs</strong>, mitjançant:
        <ul>
            <li><em>EarlyStopping</em> monitoritzant la pèrdua (<em>loss</em>) en validació, amb paciència de 10 epochs, restablint els millors pesos obtinguts.
            <li><em>ReduceLROnPlateau</em> monitoritzant la pèrdua (<em>loss</em>) en validació, factor 0,2, paciència de 5 epochs i un <em>learning rate</em> mínim de 1e-6 </i>
            <li> La mètrica de rendiment <em>accuracy</em> tant en entrenament com en validació.
        </ul>
   </li>
   <li> Gràfic de de les corbes de <em>accuracy</em> i <em>loss</em> per als conjunts d'entrenament i validació.
   <li> Finalment, avalueu el model entrenat al conjunt de test, mostrant la pèrdua (<em>loss</em>) i la precisió (<em>accuracy</em>) obtingudes en aquestes dades.
</ul>

Feu una comparació de l'efecte del <em>learning rate</em> en els 3 entrenaments basada en:
    <ul>
        <li> Número d’èpoques/temps d’entrenament.
        <li> Forma de les corbes d’entrenament.
        <li> Rendiment del model al conjunt de prova (test).
        <li> Quina taxa d’aprenentatge seleccionaries en funció del que s'ha comentat en els punts anteriors?
    </ul>

<strong> Nota: es valorarà la concisió a les preguntes (una o dues frases són suficients per a cadascuna de les respostes).
</div>

Comencem amb lr = 1e-3

In [ ]:
from timeit import default_timer as timer

# Definició funció de compilació
def get_compiled_model(lr):
    model = keras.models.clone_model(get_uncompiled_model())
    optimizer = keras.optimizers.Adam(learning_rate=lr)
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model

# Definició de callbacks
class CustomCallback(keras.callbacks.Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)
    def on_train_end(self, epoch, logs=None):
        keys = list(logs.keys())
        print("Stop training at epoch{}; got log keys: {}",epoch,format(keys))

early_stopping = keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=10, restore_best_weights=True)

reduce_lr = keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

# Definició funció per graficar accuracy i loss
def get_acc_loss_plot(lr,history):
    plt.figure(figsize=(15, 5))
    # Gràfic de loss
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label=f'Train (lr={lr})')
    plt.plot(history.history['val_loss'], label=f'Val (lr={lr})')
    
    # Gràfic d'accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label=f'Train (lr={lr})')
    plt.plot(history.history['val_accuracy'], label=f'Val (lr={lr})')
    
    plt.subplot(1, 2, 1)
    plt.title("Loss durant l'entrenament")
    plt.xlabel("Èpoques")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid()
    
    plt.subplot(1, 2, 2)
    plt.title("Accuracy durant l'entrenament")
    plt.xlabel("Èpoques")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.grid()
    
    plt.show()

In [ ]:
# Compilació del model
lr=1e-3
model1 = get_compiled_model(lr)
model1.summary()
keras.utils.plot_model(model1, "model1.png", show_shapes=True,dpi=64)

In [ ]:
# Entrenament del model
custom_callback1 = CustomCallback()

history1 = model1.fit(
    train_ds,
    validation_data=val_ds,
    epochs=100,
    batch_size=batch_size,
    callbacks=[early_stopping,reduce_lr,custom_callback1],
    verbose=1
)

In [ ]:
print(f" El model {model1.name} s'ha entrenat durant {round(sum(custom_callback1.logs),2)} ")

In [ ]:
# Grafics d'accuracy i loss
get_acc_loss_plot(lr,history1)

# Validació al conjunt de prova
result1 = model1.evaluate(test_ds)

Repetim amb lr = 1e-4

In [ ]:
# Crear un nou model sense pesos previs
lr=1e-4
model2 = get_compiled_model(lr)
print(model2)

# Entrenament del model
history2 = model2.fit(
    train_ds,
    validation_data=val_ds,
    epochs=100,
    batch_size=batch_size,
    callbacks=[early_stopping,reduce_lr,custom_callback],
    verbose=1
)

In [ ]:
print(f" El model {model2.name} s'ha entrenat durant {round(sum(custom_callback.logs),2)} ")

# Grafics d'accuracy i loss
get_acc_loss_plot(lr,history2)

# Validació al conjunt de prova
result2 = model2.evaluate(test_ds)

Acabem amb lr = 1e-5

In [ ]:
# Crear un nou model sense pesos previs
lr=1e-5
model3 = get_compiled_model(lr)
print(model3)

# Entrenament del model
history3 = model3.fit(
    train_ds,
    validation_data=val_ds,
    epochs=100,
    batch_size=batch_size,
    callbacks=[early_stopping,reduce_lr,custom_callback],
    verbose=1
)

In [ ]:
# Grafics d'accuracy i loss
get_acc_loss_plot(lr,history3)

# Validació al conjunt de prova
result3 = model3.evaluate(test_ds)

<div style="background-color: #fcf2f2; border-color: #dfb5b4; border-left: 5px solid #dfb5b4; padding: 0.5em;">
<strong>Comentaris:</strong>
    <ul>
        <li> <strong> Forma de les corbes d'entrenament </strong>: (escriu la teva resposta aquí)</li>
        <li> <strong> Número d’èpoques/temps d’entrenament </strong>: (escriu la teva resposta aquí)</li>        
        <li> <strong> Rendiment del model al conjunt de prova</strong>: (escriu la teva resposta aquí)</li>
        <li> <strong>Elecció final </strong>: (escriu la teva resposta aquí)</li>
    </ul>

<strong>Referències</strong>:
    <ul>
        <li>https://keras.io/api/callbacks/</li>
        <li>https://www.tensorflow.org/tutorials/keras/save_and_load#manually_save_weights</li>
        <li>https://keras.io/guides/training_with_built_in_methods/</li>
        <li>https://keras.io/api/models/model_training_apis/</li>
        <li>https://keras.io/guides/writing_your_own_callbacks/</li>
    </ul>
</div>

## 3. Petita xarxa convolucional (2 punts)

Ara implementarem una xarxa neuronal convolucional bàsica (CNN), que sol ser molt més eficaç per a la classificació de les imatges. Les CNN aprofiten l'estructura espacial de les dades mitjançant capes específiques que permeten extreure les característiques locals abans de la classificació.

**Arquitectura proposada**: Utilitzarem un model de Keras Sequential per a aquesta CNN. Consistirà en un bloc d’extractor de característiques i, a continuació, un classificador dens similar a l'anterior però més petit.

En aquesta secció utilitzarem les capes  [Conv2D](https://keras.io/api/layers/convolution_layers/convolution2d/),  [MaxPooling2D](https://keras.io/api/layers/pooling_layers/max_pooling2d/), [Dense](https://keras.io/api/layers/core_layers/dense/) i [Dropout](https://keras.io/api/layers/regularization_layers/dropout/) de Keras.

Es proporciona el codi del classificador ja implementat.

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;"> <strong>Exercici 3.1 [0,5 pts]:</strong> Analitza el codi següent i descriu les capes que formen el bloc extractor de característiques i el classificador.</div>

In [ ]:
# Definició del model CNN
cnn_model = keras.Sequential([
    keras.layers.InputLayer(shape=(img_height, img_width, 3)),
    Rescaling(1./255),
    Conv2D(16, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
], name="CNN_model")
cnn_model.summary()

<div style="background-color: #fcf2f2; border-color: #dfb5b4; border-left: 5px solid #dfb5b4; padding: 0.5em;">
<strong>Resposta: </strong>
<br>
 <ul>
     <li> <u>Bloc convolucional:</u> 
         <br>La primera capa normalitza els valors dels 3 canals a valors entre 0 i 1. 
         <br>La primera capa convolucional aplica 16 filtres de 3x3 en 3 canals amb s=1 i padding=1 (nombre de paràmetres = (3x3x3+1)x16 = 448), aplica la funció d'activació ReLU i redueix la dimensionalitat a la meitat amb un filtre max_pooling de 2x2. 
         <br>La segona capa convolucional aplica 32 filtres de 3x3 en 16 canals amb s=1 i padding=1 (nombre de paràmetres = (3x3x16+1)x32 = 4640), aplica la funció d'activació ReLU i redueix la dimensionalitat a la meitat amb un filtre max_pooling de 2x2.  
         <br>La tercera capa convolucional aplica 64 filtres de 3x3 en 32 canals amb s=1 i padding=1 (nombre de paràmetres = (3x3x32+1)x64 = 18496), aplica la funció d'activació ReLU i redueix la dimensionalitat a la meitat amb un filtre max_pooling de 2x2. 
     </li>
     <li> <u>Classificador final:</u>
         <br>La primera capa FC inclou una capa dropout per tal de prevenir el sobreentrenament que desactiva un 20% de les entrades (només activa en la fase d'entrenament) i una capa densa de 64 neurones actuant sobre els 64 canals (nombre de paràmetres = (64+1)*64 = 4160).
         <br>La segona capa FC inclou una capa dropout per tal de prevenir el sobreentrenament que desactiva un 50% de les entrades (només activa en la fase d'entrenament) i una capa densa de 10 neurones actuant sobre els 64 canals (nombre de paràmetres = (64+1)*10 = 650) activada amb 'softmax' per tal de donar-nos la sortida del classificador.
     </li>
 <ul>
</div>

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;"> <strong>Exercici 3.2 [0.5 pts]</strong>: Compila el model amb l'optimitzador <strong>Adam</strong> i <em>learning rate</em> = 1e-4. Entrena el model durant <strong>100 èpoques</strong>, utilitzant:
    <ul>
        <li><em>EarlyStopping</em> monitoritzant la pèrdua (<em>loss</em>) en validació, amb paciència de 10 èpoques, restaurant els millors pesos obtinguts.</li>
        <li><em>ReduceLROnPlateau</em> monitoritzant la pèrdua (<em>loss</em>) en validació, factor 0.2, paciència de 5 èpoques i un learning rate mínim de 1e-6</i>
        <li>La mètrica de rendiment <em>accuracy</em> tant en entrenament com en validació.</li>
    </ul>

Mostra una gràfica de les corbes d'accuracy i loss per als conjunts d'entrenament i validació.

Finalment, avalua el model entrenat sobre el conjunt de <test>test</test>, mostrant la pèrdua i exactitud (<it>accuracy</it>) final obtingudes en aquestes dades.

<strong>Nota: durant l'execució del codi es produirà un error.</strong>

Explica a què és degut l'error en l'execució anterior i modifica el model per solucionar-lo.

<strong> Nota: es valorarà la concisió a les preguntes (una o dues frases són suficients per a cadascuna de les respostes).
</div>

In [ ]:
# Definició funció compilació
def get_compiled_model_cnn(lr,ref_model):
    model = keras.models.clone_model(ref_model)
    optimizer = keras.optimizers.Adam(learning_rate=lr)
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model

In [ ]:
# Compilar model CNN
lr=1e-4
model4 = get_compiled_model_cnn(lr,cnn_model)
model4

In [ ]:
# # Entrenament del model CNN
# history4 = model4.fit(
#     train_ds,
#     validation_data=val_ds,
#     epochs=100,
#     batch_size=batch_size,
#     callbacks=[early_stopping,reduce_lr,custom_callback],
#     verbose=1
# )

# # Grafics d'accuracy i loss
# get_acc_loss_plot(lr,history4)

# # Validació al conjunt de prova
# result4 = model4.evaluate(test_ds)

<div style="background-color: #fcf2f2; border-color: #dfb5b4; border-left: 5px solid #dfb5b4; padding: 0.5em;">
<strong>Comentaris:</strong>
<br>
<br>
L'error es deu a què les dimensions output i target no són iguals ja que no s'ha aplanat la sortida.
    <br> <br>
<strong>Referències</strong>:
    <ul>
        <li>https://keras.io/api/layers/convolution_layers/convolution2d/</li>
        <li>https://keras.io/api/layers/pooling_layers/max_pooling2d/</li>
        <li>https://keras.io/api/layers/regularization_layers/dropout/</li>
        <li>https://keras.io/guides/serialization_and_saving/</li>
    </ul>
</div>

In [ ]:
# Escriu aquí el model corregit
cnn_model_c = keras.Sequential([
    keras.layers.InputLayer(shape=(img_height, img_width, 3)),
    Rescaling(1./255),
    Conv2D(16, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Flatten(),
    Dense(10, activation='softmax')
])
cnn_model_c.summary()

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;"> <strong>Exercici 3.3 [0.5 pt]:</strong> Torna a repetir l'exercici anterior (entrenament, mostrar gràfiques i validació en el conjunt de test) amb el model corregit. Comenta la mida del model, el nombre d'epochs/temps d'entrenament, les corbes d'entrenament i els resultats de validació.
<br><br>
<strong>NOTA: Es valorarà la concisió en la resposta a les preguntes (una o dues frases són suficients per a cadascuna de les respostes).</strong>

</div>

In [ ]:
# Compilar model CNN
lr=1e-4
model5 = get_compiled_model_cnn(lr,cnn_model_c)
model5

In [ ]:
# Entrenament del model CNN
history5 = model5.fit(
    train_ds,
    validation_data=val_ds,
    epochs=100,
    batch_size=batch_size,
    callbacks=[early_stopping,reduce_lr],
    verbose=1
)

In [ ]:
# Grafics d'accuracy i loss
get_acc_loss_plot(lr,history5)

# Validació al conjunt de prova
result5 = model5.evaluate(test_ds)

In [ ]:
# Escriu aquí el model corregit
cnn_model_d = keras.Sequential([
    keras.layers.InputLayer(shape=(img_height, img_width, 3)),
    Rescaling(1./255),
    Conv2D(16, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])
cnn_model_d.summary()

In [ ]:
# Compilar model CNN
lr=1e-4
model6 = get_compiled_model_cnn(lr,cnn_model_d)

# Entrenament del model CNN
history6 = model6.fit(
    train_ds,
    validation_data=val_ds,
    epochs=100,
    batch_size=batch_size,
    callbacks=[early_stopping,reduce_lr,custom_callback],
    verbose=1
)

In [ ]:
# Grafics d'accuracy i loss
get_acc_loss_plot(lr,history6)

# Validació al conjunt de prova
result6 = model6.evaluate(test_ds)

<div style="background-color: #fcf2f2; border-color: #dfb5b4; border-left: 5px solid #dfb5b4; padding: 0.5em;">
<strong>cnn_model_c:</strong>
<br>El model té una mida de: 283.754 paràmetres.
<br>El nombre d'epochs d'entrenament ha estat de xxx amb un temps de xxx, les corbes d'entrenament i els resultats de validació.
<br> <br>
<strong>cnn_model_d:</strong>
<br>El model té una mida de: 1.662.698 paràmetres.
<br>El nombre d'epochs d'entrenament ha estat de xxx amb un temps de xxx, les corbes d'entrenament i els resultats de validació.
<br> <br>
</div>

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;"> <strong>Exercici 3.4 [0.5 pt]:</strong> Realitza una comparativa entre els models dels exercicis 2 (amb el <em>learning rate</em> escollit) i 3 en termes de:
<ul>
  <li> <strong> Mida del model </strong> </li>
  <li> <strong> Evolució de l'entrenament </strong> </li>
  <li> <strong> Validació en el conjunt de prova </strong> </li>
  <li> <strong> Temps d'execució </strong> </li>
</ul>

<strong> Nota: es valorarà la concisió a les preguntes (una o dues frases són suficients per a cadascuna de les respostes).

</div>

<div style="background-color: #fcf2f2; border-color: #dfb5b4; border-left: 5px solid #dfb5b4; padding: 0.5em;">
<strong>Comentaris:</strong>
<ul>
  <li> <strong>Mida del model</strong>: (escriu la teva resposta aquí)</li>
  <li> <strong>Evolució de l'entrenament</strong>: (escriu la teva resposta aquí)</li>
  <li> <strong>Validació en el conjunt de prova</strong>: (escriu la teva resposta aquí)</li>
  <li> <strong>Temps d'execució</strong>: (escriu la teva resposta aquí)</li>
</ul>
</div>

## 4. Augmentació de dades (1 punt)

Tot i que la CNN entrenada és força efectiva, podem intentar millorar-ne la generalització augmentant artificialment la mida i la diversitat del conjunt d'entrenament mitjançant **tècniques d'augmentació de dades**. L'augmentació consisteix a aplicar transformacions aleatòries a les imatges (girs, rotacions, zoom, etc.) de manera que el model rebi variants de les imatges originals a cada època, simulant tenir més dades

Keras proporciona capes de preprocessament d'imatge que realitzen aquestes transformacions de manera eficient durant l'entrenament, per exemple RandomFlip, RandomRotation, RandomZoom entre d'altres.

Aquí farem servir algunes d'aquestes capes per implementar l'augment. En particular, provarem de voltejar horitzontalment les imatges i aplicar petites rotacions aleatòries.

### 4.1. Definició i visualització de l'augmentació

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;"> <strong>Exercici 4.1 [0,5 pts]:</strong> Crea un model <strong>Sequential</strong> de Keras que representi una capa d'augmentació de dades amb les següents capes:
  <ul>
      <li><code>RandomFlip</code> que voltegi aleatòriament les imatges horitzontalment.</li>
      <li><code>RandomRotation</code> amb factor de rotació de 0.1 (±10%).</li>
  </ul>
Selecciona una imatge del conjunt d'entrenament i passa-la a través d'aquest model diverses vegades, mostrant les imatges resultants per comprovar visualment les transformacions realitzades (voltejos i rotacions).
</div>

In [ ]:
# Model d'augmentació de dades
model_aug = keras.Sequential([
    keras.layers.InputLayer(shape=(img_height, img_width, 3)),
    RandomFlip(mode='horizontal'),
    RandomRotation(0.1)
])

In [ ]:
# Prendre un batch d'entrenament i obtenir una imatge
rd_class_path = os.path.join(train_path,random.choice(os.listdir(train_path)))
rd_image_path = os.path.join(rd_class_path,random.choice(os.listdir(rd_class_path)))

# Càrrega de la imatge i conversió a tensor
img = keras.preprocessing.image.load_img(rd_image_path, target_size= (img_height, img_width))
img_tensor = keras.preprocessing.image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)

# Normalitzem els valors al rang [0,1]
img_tensor /= 255.

# Grafiquem la imatge
plt.figure(figsize=(6,6))
plt.imshow(img_tensor[0])
plt.title('Imatge Original')
plt.axis('off')
plt.show()

# Aplicar augmentació diverses vegades i visualitzar
plt.figure(figsize=(12, 12))
for i in range(9):
    augmented_img = model_aug(img_tensor, training=True)
    plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_img[0].numpy())
    plt.axis("off")
plt.suptitle('Imatges Augmentades')
plt.show()

### 4.2. Entrenament de la CNN amb augmentació

Ara incorporarem la capa d’augmentació al model CNN per entrenar-la amb dades augmentades a cada època.

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;"> <strong>Exercici 4.2 [0,5 pts]:</strong> Insereix el model d'augmentació de dades creat entre la capa de <code>Rescaling</code> i la primera <code>Conv2D</code> del model CNN anterior. És a dir, modifica l'arquitectura perquè les imatges d'entrada, després de ser reescalades, passin per les capes de <em>flip</em> i <em>rotation</em> aleatòries, i després continuïn per la CNN. A continuació, compila i entrena el nou model CNN augmentat seguint les mateixes indicacions que en l'exercici anterior, <strong>excepte</strong> que aquesta vegada farem servir un <strong>learning rate</strong> inicial més gran (1e-3). Mantingues EarlyStopping, ReduceLROnPlateau, 100 èpoques, etc. Després avalua el model final al conjunt de test.

Comenta les diferències amb el model sense augmentació en termes de:

<ul>
    <li> <strong> Evolució de les corbes d'entrenament </strong> </li>
    <li> <strong> Validació en el conjunt de prova </strong> </li>
    <li> <strong> Temps d'entrenament </strong> </li>
    <li> <strong> Learning rate </strong> </li>
</ul>

<strong> Nota: es valorarà la concisió a les preguntes (una o dues frases són suficients per a cadascuna de les respostes).

</div>

In [ ]:
# Model CNN amb augmentació de dades
cnn_model_aug = keras.Sequential([
    keras.layers.InputLayer(shape=(img_height, img_width, 3)),
    Rescaling(1./255),
    RandomFlip(mode='horizontal'),
    RandomRotation(0.1),
    Conv2D(16, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])
cnn_model_aug.summary()

In [ ]:
# Compilació de la xarxa
lr=1e-3
model7 = get_compiled_model_cnn(lr,cnn_model_aug)

In [ ]:
# Entrenament del model CNN
history7 = model7.fit(
    train_ds,
    validation_data=val_ds,
    epochs=100,
    batch_size=batch_size,
    callbacks=[early_stopping,reduce_lr,custom_callback],
    verbose=1
)

In [ ]:
# Resultats
# Grafics d'accuracy i loss
get_acc_loss_plot(lr,history6)

# Validació al conjunt de prova
result7 = model7.evaluate(test_ds)

<div style="background-color: #fcf2f2; border-color: #dfb5b4; border-left: 5px solid #dfb5b4; padding: 0.5em;">
<strong> Comentaris: </strong>
<br> <br>

<ul>
  <li> <strong> Evolució de les corbes d'entrenament</strong>: (escriu la teva resposta aquí) </li>
  <li> <strong> Validació en el conjunt de prova</strong>: (escriu la teva resposta aquí) </li>
  <li> <strong> Temps d'entrenament</strong>: (escriu la teva resposta aquí) </li>
  <li> <strong> Learning rate</strong>: (escriu la teva resposta aquí) </li>
</ul>
<br>
<strong> Referències: </strong>
<br> <br>

<ul>
    <li>https://keras.io/api/layers/preprocessing_layers/image_augmentation/random_flip/</li>
    <li>https://keras.io/api/layers/preprocessing_layers/image_augmentation/</li>
    <li>https://keras.io/api/layers/preprocessing_layers/image_augmentation/random_rotation/</li>
    <li>https://gac6.medium.com/visualizing-data-augmentations-from-keras-image-data-generator-44f040aa4c9f</li>
<li>chatgpt.com</li>
</ul>

</div>

## 5. Superresolució d’imatges amb CNNs (2 pts)

Fins ara hem treballat en un problema de “classificació”. Als apartats restants abordarem un problema diferent però relacionat amb la visió per computador: la **superresolució d'imatges**. La superresolució consisteix a generar una imatge d'alta resolució (HR) a partir d'una de baixa resolució (LR), intentant recuperar o inferir els detalls perduts en reduir la imatge. És un problema d'aprenentatge supervisat on el model aprèn una transformació imatge->imatge.

Farem servir novament la base de dades Imagenette per crear exemples d'entrenament: a partir de cada imatge original (320px) generarem una versió reduïda (p. ex. 80px) que servirà com a entrada, tenint com a sortida esperada la imatge original. Així, el model aprendrà a mapejar de baixa a alta resolució. En lloc d'una xarxa convolucional per a la classificació, necessitarem una xarxa capaç de processar una imatge d'entrada i produir una imatge de sortida. Les **capes transposades de convolució** ([Conv2DTranspose](https://keras.io/api/layers/convolution_layers/convolution2d_transpose/)) o les tècniques de upsampling són les peces clau per a aquests models, ja que permeten augmentar les dimensions espacials de les dades.

A continuació, crearem el conjunt de dades per a superresolució i entrenarem una **CNN de superresolució simple**.

### 5.1. Preparació de dades de LR/HR

Primer generarem els parells d'imatges d'entrenament i de validació per a superresolució. Partirem de les imatges originals d'entrenament (i validació) a la seva resolució completa i, per simplicitat, les redimensionarem a una mida fixa de 320×320 (ignorant la relació d'aspecte original, similar al fet en classificació) per utilitzar-les com a imatges de referència d'alta resolució (HR), i les reduirem a 1/4 de la seva mida (aprox). Per realitzar ambdues transformacions (ajust de les imatges originals a 320x320 i la seva reducció a 80x80) utilitzarem mètodes d'[interpolació bicúbica](https://es.wikipedia.org/wiki/Interpolaci%C3%B3n_bic%C3%BAbica) per simular imatges degrades suaument.


Procedim a obtenir les rutes d'imatges d'entrenament i validació, després creem un Dataset aplicant la funció de mapeig que realitza la lectura i la transformació:

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;"> <strong>Exercici 5.1.1 [0,5 pt]: </strong>El codi següent prepara un nou conjunt de dades a partir de les imatges d'entrenament originals. Cada exemple consistirà en una imatge de baixa resolució (LR) i la corresponent imatge d'alta resolució (HR). Concretament, pren les imatges de <code>/train/</code> (per crear els subconjunts d'entrenament i validació) a la seva mida completa (320px) com a HR, i genera imatges LR reduint-les a 1/4 de la seva mida lineal (80×80). Es demana:
<ul>
    <li> Completa els comentaris del codi en aquells llocs on està marcat (# Posar Comentari).
</ul>
</div>

In [ ]:
# Funció per al preprocessat d'imatges per al problema de SR
def preprocess_image_for_sr(filepath):
    # Càrrega i descodificació del fitxer
    img = tf.io.read_file(filepath)
    img = tf.image.decode_jpeg(img, channels=3)
    # Conversió a tensor float32 (valors entre 0 i 1)
    img = tf.image.convert_image_dtype(img, tf.float32)
    # Reescalat mitjançant interpolació bicúbica
    hr = tf.image.resize(img, [320, 320], method=tf.image.ResizeMethod.BICUBIC)
    lr = tf.image.resize(img, [80, 80], method=tf.image.ResizeMethod.BICUBIC)
    # Normalització dels valors del tensor
    hr = tf.clip_by_value(hr, 0.0, 1.0)
    lr = tf.clip_by_value(lr, 0.0, 1.0)
    return lr, hr

# Obtenim els conjunts d'entrenament i validació
train_files = []
val_files = []
for cls in train_classes:
    cls_files = random.shuffle((train_path/cls).glob("*.*"))
    # Separem els conjunts d'entrenament (80%) i validació (20%)
    split_idx = int(len(cls_files) * 0.2)
    val_files += [str(p) for p in cls_files[:split_idx]]
    train_files += [str(p) for p in cls_files[split_idx:]]
random.shuffle(train_files)
random.shuffle(val_files)

# Creem els datasets d'entrenament i validació
train_sr_ds = tf.data.Dataset.from_tensor_slices(train_files) # Rutes
.map(preprocess_image_for_sr, num_parallel_calls=tf.data.AUTOTUNE) # LR, HR
.batch(32) # Agrupació lots de 32
.prefetch(tf.data.AUTOTUNE) # Optimització processament
val_sr_ds   = tf.data.Dataset.from_tensor_slices(val_files)
.map(preprocess_image_for_sr, num_parallel_calls=tf.data.AUTOTUNE)
.batch(32)
.prefetch(tf.data.AUTOTUNE)

Procedim a verificar la base de dades:

In [ ]:
# Verificació d'un parell LR-HR
for lr_batch, hr_batch in train_sr_ds.take(1):
    lr_img = lr_batch[0].numpy()
    hr_img = hr_batch[0].numpy()
    print("LR shape:", lr_img.shape, "HR shape:", hr_img.shape)
    print("LR pixel range:", lr_img.min(), "-", lr_img.max())
    print("HR pixel range:", hr_img.min(), "-", hr_img.max())
    break
print(f"El nombre d'imatges en el conjunt d'entrenament és {len(train_files)}")
print(f"El nombre d'imatges en el conjunt de validació és {len(val_files)}")

<div style = "background-color: #edf7ff; border-color: #7c9dbf; frontera-esquerra: 5px sòlid #7c9dbf; encoixinat: 0,5EM;"> <strong> Exercici 5.1.2 [0,5 pt]: </strong> Respon a les preguntes següents:
<ul>
 <li>A què es pot deure l'ús de la funció 'clip_by_value' al codi anterior?</li>
 <li>Com es divideix la base de dades? Quin directori s'utilitza per crear-la? Quins subconjunts es creen? Quin percentatge d'imatges s'assigna a cada conjunt? I en nombre d'imatges?</li>
 <li>Per a cada element del conjunt de dades, quin format tenen les dades que farem servir per entrenar el model? i les etiquetes?</li>
</ul>
<strong> Nota: es valorarà la concisió a les preguntes (una o dues frases són suficients per a cadascuna de les respostes).
</div>

<div style="background-color: #fcf2f2; border-color: #dfb5b4; border-left: 5px solid #dfb5b4; padding: 0.5em;">
<strong> Comentaris: </strong>
<br> <br>
 <ul>
    <li> <strong> Funció 'clip_by_value' </strong>: La funció ens permet corregir possibles valors fora del rang [0,1] generats en la interpolació bicúbica. </li>
    <li> <strong> Subconjunts de la base de dades </strong>: La base de dades es divideix en un conjunt d'entrenament que conté un 80% de les dades (7578 imatges) i un conjunt de validació que conté el 20% restant (1891 imatges). </li>
    <li> <strong> Format de les dades i etiquetes </strong>: Les dades d'entrada (LR) tenen una mida de 80x80 píxels representats en un tensor (80,80,3). Les etiquetes (HR) tenen una mida de 320x320 píxels representats en un tensor (320,320,3) </li>
</ul>
<strong> Referències: </strong>
<br> <br>
 <ul>
     <li>https://www.tensorflow.org/api_docs/python/tf/clip_by_value</li>
     <li>chatgpt.com</li>
</ul>
</div>

### 5.2. Implementació del model de Superresolució

Ara definim un model CNN per a Superresolució. Optarem per una arquitectura senzilla amb [UpSampling2D](https://keras.io/api/layers/reshaping_layers/up_sampling2d/) per escalar la imatge gradualment. Implementem el model:

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;"> <strong> Exercici 5.2 [1 pt]: </strong> Entrenarem un model CNN per superresolució 4× (de 80px a 320px). Per fer això:
    <ul>
        <li> Implementa un model de superresolució a Keras. Podeu utilitzar l’API funcional o Sequential. Suggeriment d'arquitectura simple:
            <ul>
                <li> Entrada amb forma (80,80,3).
                <li> Diverses capes Conv2D per extreure les característiques de les imatges de baixa resolució.
                <li> Capa de <code>Upsampling2D</code> (factor=2) o <code>Conv2DTranspose</code> amb <code>stride=2</code> per doblar l'amplada i alçada (80 -> 160).
                <li> Una altra capa Conv2D per refinar, seguida d'una altra capa upsampling 2× (160 -> 320).
                <li> Capes finals Conv2D per reconstruir la imatge de sortida 320×320×3. (Podeu utilitzar l'activació lineal o RELU per a la sortida; considereu que busquem valors de píxel entre 0-1 o 0-255).
            </ul> </li>
        <li> Compileu el model amb una funció de pèrdua adequada per a imatges (per exemple, MSE) i sense mètriques de precisió. Entreneu-lo durant ~ 50-100 epochs, amb EarlyStopping (paciència 5-10, monitorització de val_loss) per assegurar-vos que no sobreentreneu.
        <li> Comentar els resultats de l'entrenament. Concretament:
            <ul>
                <li> Corbes d'entrenament.
                <li> El temps d'entrenament vs Tamany del model.
            </ul>                
        </li>
    </ul>
<strong> Nota: es valorarà la concisió a les preguntes (una o dues frases són suficients per a cadascuna de les respostes).
</div>

In [ ]:
# Model de Superrsolució (80px -> 320px)
# Model de Superresolució (80px -> 320px)
inputs = keras.Input(shape=(80, 80, 3))

# Extracció de característiques
x = layers.Conv2D(32, 3, padding='same', activation='relu')(inputs)
x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)

# Primera fase d'upsampling (80 -> 160)
x = layers.UpSampling2D(size=2)(x)
x = layers.Conv2D(128, 3, padding='same', activation='relu')(x)

# Segona fase d'upsampling (160 -> 320)
x = layers.UpSampling2D(size=2)(x)
x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)

# Refinament i reconstrucció
x = layers.Conv2D(32, 3, padding='same', activation='relu')(x)
outputs = layers.Conv2D(3, 3, padding='same', activation='sigmoid')(x)  # Sortida entre 0-1

In [ ]:
# Compilació de la xarxa
model = keras.Model(inputs, outputs)
model.compile(optimizer='adam', loss='mse')

# Resum del model
model.summary()

In [ ]:
# Entrenament de la xarxa


In [ ]:
# Resultats


<div style="background-color: #fcf2f2; border-color: #dfb5b4; border-left: 5px solid #dfb5b4; padding: 0.5em;">
<strong> Comentaris: </strong>
<br> <br>
<ul>
    <li> <strong> Corbes d’entrenament </strong>: (escriviu la vostra resposta aquí) </li>
    <li> <strong> Temps d'entrenament vs Tamany del model </strong>: (escriviu la vostra resposta aquí) </li>
</ul>
</div>

## 6. Inferència, mètriques i visualització dels resultats (1 punt)

Amb el model de superresolució entrenat, avaluarem el seu rendiment tant quantitativament (amb mètriques) com qualitativament (visualització d’imatges). Per fer -ho, realitzarem la inferència: és a dir, prendre imatges LR del conjunt de test, passar-les pel model <code>sr_model</code> per obtenir imatges supers resoltes (SR) i comparar-les amb les imatges HR originals. Calcularem la mètrica [PSNR (Peak Signal-to-Notise Ratio)](https://es.wikipedia.org/wiki/psnr) per quantificar la qualitat.

El PSNR es mesura en decibels (dB) i els valors superiors indiquen una major similitud amb la imatge original (per exemple,> 30 dB sol indicar una qualitat de reconstrucció molt bona).

També visualitzarem alguns exemples, on mostraren la imatge de baixa resolució (LR), la super resolta per la CNN (SR) i l’original d’alta resolució (HR), per inspeccionar els detalls amb l’ull nu.

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;"> <strong>Exercici 6 [1 pt]:</strong> Realitza la inferència del model de superresolució sobre diverses imatges de test:
<ul>
  <li>Per a cada imatge de test, obté la seva versió de baixa resolució (LR) i genera la imatge superresolta (SR) amb el model.</li>
  <li>Calcula la mètrica PSNR comparant la imatge SR amb la HR original per a tot el conjunt de test.</li>
  <li>Mostra visualment alguns exemples, incloent-hi la imatge d'entrada LR (pots ampliar-la per mostrar-la de la mateixa mida utilitzant el mètode <code>tf.image.resize()</code> juntament amb <code>method=tf.image.ResizeMethod.NEAREST_NEIGHBOR</code>), la imatge generada pel model (SR) i la imatge HR original, per poder comparar la qualitat visualment a part del valor de la PSNR.</li>
</ul>
Comentar els resultats obtinguts:
<ul>
    <li> Quins valors de PSNR s’obtenen? (Comenta tant els exemples mostrats com el valor mitjà del total del conjunt de prova) </li>
    <li> Les imatges creades amb superresolució són nítides o borroses?
    <li> Quins tipus de detalls o errors s’observen?
</ul>
<strong> Nota: es valorarà la concisió a les preguntes (una o dues frases són suficients per a cadascuna de les respostes).  
</div>

In [ ]:
# Preparar conjunt de test per a SR (a partir del directori /val )


In [ ]:
# Llista per emmagatzemar la PSNR de cada imatge

# Recórrer el dataset de test (batch size = 1)

 # Generar la imatge de superresolució a partir de la imatge LR

 # Treure la dimensió de batch i assegurar que els valors estiguin a [0,1]

 # Convertir la imatge HR a array de numpy (llevem la dimensió de batch)

 # Calcular la PSNR entre la imatge SR generada i la imatge HR original

 # Emmagatzemar el valor de PSNR a la llista


# Calcular la PSNR mitjana del conjunt de test
avg_psnr =
print("PSNR mitjana al conjunt de test: {:.2f} dB".format(avg_psnr))

In [ ]:
import math

# Escollir algunes imatges de test aleatòries per a visualització

# Per a cada imatge

 # Llegir i processar una imatge de test

 # Generar superresolució

 # Calcular PSNR

 # Preparar imatges per visualitzar de la mateixa mida

 # Les imatges estan a [0,1], escalar a [0,255] per mostrar correctament

 # Mostrar les imatges



<div style="background-color: #fcf2f2; border-color: #dfb5b4; border-left: 5px solid #dfb5b4; padding: 0.5em;">
<strong> Comentaris: </strong>
<br> <br>
   Anàlisi dels resultats del model SR:
   <ul>
       <li> <strong> PSNR </strong>: (escriviu la vostra resposta aquí) </li>
       <li> <strong> Qualitat visual </strong>: (escriviu la vostra resposta aquí) </li>
       <li> <strong> Errors </strong>: (escriviu la vostra resposta aquí) </li>
   </ul>
        
   (Escriviu la vostra resposta aquí)
</div>

## 7. Model pre-entrenat en superresolució (1 punt)

Els avenços recents en superresolució han produït arquitectures més complexes (p. ex., basades en xarxes generatives adversàries) que aconsegueixen resultats notablement millors, a costa d'un entrenament costós. Un d'aquests models és ESRGAN ([Enhanced Super-Resolution GAN per Xintao Wang et al.](https://arxiv.org/abs/1809.00219)), entrenat en grans bases de dades d'imatges HD (com DIV2K) per aconseguir superresolució 4x de gran fidelitat.

Farem servir un model pre-entrenat d'ESRGAN disponible via [TensorFlow Hub](https://www.tensorflow.org/hub/tutorials/image_enhancing)

Aquest model ha estat entrenat al conjunt DIV2K (imatges d'alta qualitat) amb degradació bicúbica, per la qual cosa està especialitzat en produir imatges 4× més grans amb detall notable.

Farem servir ESRGAN per aplicar superresolució a les mateixes imatges de test i compararem els resultats amb el nostre model implementat a l'exercici 5.

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;"> <strong> Exercici 7 [1 pt]:</strong> Utilitza un model pre-entrenat de superresolució (p. ex. ESRGAN 4×) per millorar les imatges de baixa resolució. En concret:
  <ul>
      <li>Carrega el model pre-entrenat des de TensorFlow Hub (usa l'URL proporcionada, per exemple <code>"https://tfhub.dev/captain-pool/esrgan-tf2/1"</code>).</li>
      <li>Per a una selecció d'imatges de test de baixa resolució (80×80), obteniu la superresolució 4× amb aquest model.</li>
      <li>Calcula la PSNR de les imatges generades pel model pre-entrenat (en el total del conjunt de test) i compara la seva mitjana amb la PSNR mitjana obtinguda pel teu model.</li>
      <li>Visualitza les imatges de sortida del model pre-entrenat al costat de les del teu model i les originals, per comparar visualment la qualitat. Compara també la PSNR d'aquestes imatges.</li>
  </ul>
Discuteix les diferències observades:
  <ul>
      <li>El model pre-entrenat obté millors mètriques?</li>
      <li>El model pre-entrenat produeix imatges més nítides i properes a la realitat?</li>
      <li>En quins detalls es nota la millora?</li>
  </ul>
</div>

In [ ]:
import tensorflow_hub as hub

# Carreguem el model ESRGAN des de TF-Hub:
esrgan = hub.load("https://tfhub.dev/captain-pool/esrgan-tf2/1")
print("Model ESRGAN carregat.")

In [ ]:
# Llista per emmagatzemar la PSNR de cada imatge usant ESRGAN

# Recórrer el dataset de test (batch size = 1)

 # Preparar la imatge LR per ESRGAN:
 # ESRGAN espera entrada en rang [0,255] com a float32, i test_sr_ds té imatges a [0,1]

 # Generar la imatge de superresolució amb ESRGAN

 # Treure la dimensió de batch i normalitzar la sortida a [0,1]

 # Obtenir la imatge HR corresponent (llevem la dimensió de batch)

 # Calcular la PSNR entre la imatge generada per ESRGAN i la imatge HR original


# Calcular la PSNR mitjana en el conjunt de test amb ESRGAN
avg_psnr_esrgan =
print("PSNR mitjana en el conjunt de test amb ESRGAN: {:.2f} dB".format(avg_psnr_esrgan))

In [ ]:
# Apliquem ESRGAN a les mateixes imatges d'exemple que fem servir amb el nostre model

# Per a cada imatge

 # Preparar la imatge LR de test

 # Convertir sortida a [0,1] float

 # Calcular PSNR comparat amb HR

 # Visualitzar comparativa


<div style="background-color: #fcf2f2; border-color: #dfb5b4; border-left: 5px solid #dfb5b4; padding: 0.5em;">
<strong> Comentaris: </strong>
<ul>
    <li> <strong> Mètrica (PSNR) </strong>: (escriviu la vostra resposta aquí) </li>
    <li> <strong> Qualitat visual </strong>: (escriviu la vostra resposta aquí) </li>
    <li> <strong> Detalls </strong>: (escriviu la vostra resposta aquí) </li>
</ul>  
</div>

## 8. Comparació i conclusions (0,5 pts)

En aquesta pràctica hem explorat tant la classificació d’imatges amb xarxes neuronals (denses vs convolucionals) com la superresolució amb CNNs, aplicant-ho tot sobre la base de dades d’Imagenette.

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;"> <strong> Exercici 8 [0,5 pts]: </strong> Fes un resum dels punts apresos en aquest PAC. Concretament, resumeix en una frase el que has après de cadascun dels punts enumerats a continuació: <ul>
    <li> <strong> ANN vs CNN: </strong> </li>
    <li> <strong> Regularització i augmentació: </strong> </li>
    <li> <strong> Superresolució: </strong> </li>
    <li> <strong> Transfer learning: </strong> </li>
</ul>
Valoració final
</div>

<div style="background-color: #fcf2f2; border-color: #dfb5b4; border-left: 5px solid #dfb5b4; padding: 0.5em;">
<strong> Comentaris: </strong>
<br> <br>
Conclusions finals:
<ul>
    <li>
        <strong> ANN vs CNN: </strong> (escriviu la vostra resposta aquí)
    </li>
    <li>
        <strong> Regularització i augmentació: </strong> (escriviu la vostra resposta aquí)
    </li>
    <li>
        <strong> Superresolució: </strong> (escriviu la vostra resposta aquí)
    </li>
    <li>
        <strong> Transfer Learning: </strong> (escriviu la vostra resposta aquí)
    </li>
</ul>


  (Escriviu la vostra resposta aquí)
<br> <br>
</div>